# Text-processing with Tkinter UI for NWN logs

## Import modules

In [301]:
import os
import pandas as pd
import re
import tkinter as tk
from tkinter import Tk
from tkinter import font
from tkinter import filedialog
from PIL import ImageTk, Image
import PIL
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Text-processing functions

In [302]:
def xp_per_rest(input_file):
    pattern_xp_gained = r'Experience Points Gained\:\s\s(\d{1,4})'
    pattern_bonus_xp = r'Bonus Experience\:\s(\d{1,4})'
    pattern_rest = r'Done resting\.'

    xp_list = []
    final_list = []

    with open(input_file, 'r') as text_file:
        for line in text_file.readlines():
            if 'Done resting.' in line and len(xp_list) != 0: #Reset at rest, new cycle starts.
                final_list.append(sum(xp_list))
                xp_list = []
            elif 'Bonus Experience' in line: #Assign - value to bonus, as it will be accounted for, but we want to 0 it.
                xp_list.append(int(re.search(pattern_bonus_xp, line).group(1))*-1)
            elif 'Experience Points Gained:' in line:
                xp_list.append(int(re.search(pattern_xp_gained, line).group(1)))
    
    final_list = '\n'.join([str(i) for i in final_list]) #Convert list elements to string
    
    return print('XP per rest cycle:\n'+final_list+'\n') #Output them in new line with title added

In [315]:
def damage_data(input_file):
    pattern_damager = r'\]\s.*?\]\s(.*)\sdamages'
    pattern_damaged = r'damages\s(.*)\:'
    pattern_damage_done = r'\:\s(\d{1,3})\s\('

    end_list = []

#Construct 3-element lists (damager, damaged, damage) and add to list of lists           
    for line in input_file.readlines():
        if 'damages' in line:
            end_list.append(
                [
                    re.search(pattern_damager, line).group(1),
                    re.search(pattern_damaged, line).group(1),
                    re.search(pattern_damage_done, line).group(1)
                ]
            )

    return end_list

In [304]:
def generate_damage_table(txt_or_folder):
    if os.path.isdir(txt_or_folder): #process in loop with 2 extra fields added to DF if a folder is picked
        damage_df = pd.DataFrame(columns=['damager', 'damaged', 'damage_done', 'date', 'quest'])
        for file in os.listdir(txt_or_folder):
            if file.endswith(".txt"):
                with open(txt_or_folder+'/'+file, 'r') as text_file:
                    try:
                        date = re.search(r'(\d{8})', file).group(1) #yyyymmdd format at start of filename
                        quest = re.search(r'\_(.*)\.txt', file).group(1) #from end of filename, following '_'
                    except:
                        date = datetime.now().strftime("%Y%m%d")
                        quest = 'N/A'

                    end_list = damage_data(text_file) #call the first function here
                    df = pd.DataFrame(end_list, columns=['damager', 'damaged', 'damage_done']).assign(date=date,
                                                                                                      quest=quest)
                    damage_df = damage_df.append(df)

        damage_df = damage_df.astype({'damage_done':'int', 'date':'datetime64'})
        damage_df.sort_values(by=['quest', 'damage_done'], ascending=False, inplace=True)
        
    else: #if not a folder, meaning if standalone txt was picked
        damage_df = pd.DataFrame(columns=['damager', 'damaged', 'damage_done'])
        if txt_or_folder.endswith(".txt"):
            with open(txt_or_folder, 'r') as text_file:
                end_list = damage_data(text_file)
                damage_df = pd.DataFrame(end_list, columns=['damager', 'damaged', 'damage_done'])
                damage_df = damage_df.astype({'damage_done':'int'})
                damage_df.sort_values(by=['damage_done'], ascending=False, inplace=True)

    if len(end_list) != 0: #save to excel if the list was not empty, meaning wrong txt was used
        timestamp = datetime.now().strftime("%Y.%m.%d_%H%M%S")
        damage_df.to_excel(r'./output_files/output_'+timestamp+'.xls', index=False)
   
    return damage_df

## Interface with Tkinter

In [308]:
#Functions for the buttons to pick a folder or a txt file

def get_txt():
    filename = filedialog.askopenfilename()
    return filename


def get_folder():
    folder = filedialog.askdirectory(title='Select Folder')
    return folder

In [312]:
#Basic UI

app = tk.Tk()

HEIGHT = 500
WIDTH = 889
newsize=(HEIGHT,WIDTH)

C = tk.Canvas(app, height=HEIGHT, width=WIDTH)

#Add background image
try:
    background_image = Image.open(r'./background.gif')
    photo_image = ImageTk.PhotoImage(background_image)
    label = tk.Label(app, image=photo_image)
    label.place(x=0, y=0, relwidth=1, relheight=1)
except:
    None

frame = tk.Frame(app,  bg='#FCF3CF', bd=5)
frame.place(relx=0.5, rely=0.025, relwidth=0.3, relheight=0.1, anchor='n')

get_file_btn = tk.Button(frame, text='Get Txt', font=font.Font(size=10), fg='#FCF3CF', bg='#AF601A', 
                                                        command=lambda: generate_damage_table(get_txt()))
get_file_btn.place(relx=0.1, relheight=1, relwidth=0.35)
get_folder_btn = tk.Button(frame, text='Get Folder', font=font.Font(size=10), fg='#FCF3CF', bg='#AF601A', 
                                                        command=lambda: generate_damage_table(get_folder()))
get_folder_btn.place(relx=0.55, relheight=1, relwidth=0.35)

lower_frame = tk.Frame(app, bg='#641E16', bd=3)
lower_frame.place(relx=0.5, rely=0.6, relwidth=0.75, relheight=0.35, anchor='n')

instr_text ='''
This programme processes NWN log files and saves the output in the "output_files" folder in excel format.

By clicking the "Get Txt" button and selecting a txt log file, you can process a standalone file.

By clicking the "Get Folder" button and selecting a folder, you can process multiple files at once and
also add two extra columns. The recommended file naming convention is "yyyymmdd_quest.txt", for
example "20201231_drider.txt"
'''

instructions = tk.Label(lower_frame, anchor='nw', justify='left', text=instr_text, wraplengt=600)
instructions.config(font=font.Font(size=10), fg='#AF601A', anchor="center", bg='#FCF3CF')
instructions.place(relwidth=1, relheight=1)

C.pack()

if __name__ == '__main__':
    app.mainloop()

In [70]:
damage_df

,damager,damaged,damage_done,date,quest
1361,Holiester,Pyro,204,2020-12-07,xptest
1440,Holiester,Duergar Matriarch,190,2020-12-07,xptest
1370,Holiester,Pyro,188,2020-12-07,xptest
1458,Holiester,Duergar Matriarch,187,2020-12-07,xptest
1345,Holiester,Pyro,185,2020-12-07,xptest
1353,Holiester,Pyro,182,2020-12-07,xptest
1332,Holiester,Pyro,159,2020-12-07,xptest
1378,Holiester,Pyro,144,2020-12-07,xptest
1360,Holiester,Pyro,82,2020-12-07,xptest
1358,Holiester,Pyro,77,2020-12-07,xptest


In [72]:
damage_df.groupby(['damager']).sum().sort_values('damage_done', ascending=False)

,damage_done
damager,
Morrico,32892
Lea 'Diamondsoul',11552
Holiester,8872
Hana Haley,6995
Greg the Nasty,6298
Drogo,3853
Arianell,3510
IronCat,1528
Sacred,1520


In [248]:
xp_per_rest('./input_files/20201207_xptest.txt')

XP per rest cycle:
1270
8
744
1504

